In [4]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.2/387.2 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 57.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.2/201.2 kB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 

In [ ]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Define columns
columns = {0: 'text', 1: 'ner'}

# Directory where the data resides
data_folder = './'

# Corpus with column format
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file='train.txt',
                              test_file='test.txt',
                              dev_file='dev.txt')

2023-12-15 20:20:37,508 Reading data from .
2023-12-15 20:20:37,510 Train: train.txt
2023-12-15 20:20:37,511 Dev: dev.txt
2023-12-15 20:20:37,514 Test: test.txt


In [ ]:
print(corpus.train[0].to_tagged_string())

Sentence[2]: "In certain" → ["In certain"/Requirement]


In [ ]:
# Make tag dictionary from the corpus
tag_dictionary = corpus.make_label_dictionary(label_type="ner")
tag_dictionary

2023-12-15 20:20:45,239 Computing label dictionary. Progress:


0it [00:00, ?it/s]
5343it [00:00, 51289.94it/s]

2023-12-15 20:20:45,395 Dictionary created for label 'ner' with 4 values: Requirement (seen 6181 times), Application (seen 1755 times), Exception (seen 729 times), Selection (seen 566 times)


In [ ]:
# Assuming 'tag_dictionary' is your tag dictionary
for tag in tag_dictionary.idx2item:
    print(tag)

b'Requirement'
b'Application'
b'Exception'
b'Selection'


In [ ]:
# Initialize embeddings
embedding_types = [
    WordEmbeddings('glove'),
    # other embeddings if needed
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

# Initialize sequence tagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type='ner',
                                        rnn_layers = 2,
                                        tag_format = "BIO",
                                        use_crf=True)

# Initialize trainer
trainer: ModelTrainer = ModelTrainer(tagger, corpus)

2023-12-15 20:38:37,273 SequenceTagger predicts: Dictionary with 9 tags: O, B-Requirement, I-Requirement, B-Application, I-Application, B-Exception, I-Exception, B-Selection, I-Selection


In [ ]:

trainer.train(
    base_path='output',
    learning_rate=0.1,  # Initial learning rate
    mini_batch_size=32,
    max_epochs=150,
    anneal_factor=0.8,
    min_learning_rate=0.009
)


2023-12-15 20:38:43,001 ----------------------------------------------------------------------------------------------------
2023-12-15 20:38:43,003 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings(
      'glove'
      (embedding): Embedding(400001, 100)
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=100, out_features=100, bias=True)
  (rnn): LSTM(100, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=512, out_features=11, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)"
2023-12-15 20:38:43,004 ----------------------------------------------------------------------------------------------------
2023-12-15 20:38:43,006 Corpus: 5343 train + 535 dev + 534 test sentences
2023-12-15 20:38:43,007 ----------------------------------------------------------------------------------------------------
2023-12-

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:84: UserWarning: There should be no best model saved at epoch 1 except there is a model from previous trainings in your training folder. All previous best models will be deleted.
  warnings.warn(


2023-12-15 20:38:43,898 epoch 1 - iter 16/167 - loss 1.87420954 - time (sec): 0.86 - samples/sec: 4926.46 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:44,677 epoch 1 - iter 32/167 - loss 1.46238043 - time (sec): 1.64 - samples/sec: 4980.03 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:45,616 epoch 1 - iter 48/167 - loss 1.25883287 - time (sec): 2.58 - samples/sec: 4737.00 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:46,497 epoch 1 - iter 64/167 - loss 1.10011219 - time (sec): 3.46 - samples/sec: 4723.11 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:47,266 epoch 1 - iter 80/167 - loss 1.03836344 - time (sec): 4.23 - samples/sec: 4751.47 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:48,413 epoch 1 - iter 96/167 - loss 0.97525510 - time (sec): 5.38 - samples/sec: 4418.08 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:49,411 epoch 1 - iter 112/167 - loss 0.92454881 - time (sec): 6.37 - samples/sec: 4324.92 - lr: 0.100000 - momentum: 0.000000
2023-

100%|██████████| 9/9 [00:00<00:00, 12.58it/s]


2023-12-15 20:38:52,667 DEV : loss 0.5595788955688477 - f1-score (micro avg)  0.3074
2023-12-15 20:38:52,688  - 0 epochs without improvement
2023-12-15 20:38:52,689 saving best model
2023-12-15 20:38:53,385 ----------------------------------------------------------------------------------------------------
2023-12-15 20:38:54,245 epoch 2 - iter 16/167 - loss 0.47160775 - time (sec): 0.86 - samples/sec: 4597.53 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:54,956 epoch 2 - iter 32/167 - loss 0.46830095 - time (sec): 1.57 - samples/sec: 4976.62 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:55,811 epoch 2 - iter 48/167 - loss 0.46629544 - time (sec): 2.42 - samples/sec: 4985.50 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:56,558 epoch 2 - iter 64/167 - loss 0.47595327 - time (sec): 3.17 - samples/sec: 4990.64 - lr: 0.100000 - momentum: 0.000000
2023-12-15 20:38:57,216 epoch 2 - iter 80/167 - loss 0.47951848 - time (sec): 3.83 - samples/sec: 5103.03 - lr: 0.100000 - m

...

2023-12-15 20:59:37,259 DEV : loss 0.07139483094215393 - f1-score (micro avg)  0.9151
2023-12-15 20:59:37,279  - 4 epochs without improvement (above 'patience')-> annealing learning_rate to [0.008589934592000005]
2023-12-15 20:59:37,281 ----------------------------------------------------------------------------------------------------
2023-12-15 20:59:37,283 learning rate too small - quitting training!
2023-12-15 20:59:37,284 ----------------------------------------------------------------------------------------------------
2023-12-15 20:59:37,286 Saving model ...


2023-12-15 20:59:38,035 Done.
2023-12-15 20:59:38,037 ----------------------------------------------------------------------------------------------------
2023-12-15 20:59:38,039 Loading model from best epoch ...
2023-12-15 20:59:38,678 SequenceTagger predicts: Dictionary with 11 tags: O, B-Requirement, I-Requirement, B-Application, I-Application, B-Exception, I-Exception, B-Selection, I-Selection, <START>, <STOP>


100%|██████████| 9/9 [00:00<00:00, 13.56it/s]

2023-12-15 20:59:39,481 
Results:
- F-score (micro) 0.8925
- F-score (macro) 0.9041
- Accuracy 0.8457

By class:
              precision    recall  f1-score   support

 Requirement     0.8619    0.9030    0.8819       608
 Application     0.9396    0.8550    0.8953       200
   Selection     0.9802    0.9519    0.9659       104
   Exception     0.8846    0.8625    0.8734        80

   micro avg     0.8898    0.8952    0.8925       992
   macro avg     0.9166    0.8931    0.9041       992
weighted avg     0.8918    0.8952    0.8927       992

2023-12-15 20:59:39,482 ----------------------------------------------------------------------------------------------------


{'test_score': 0.892462311557789}

In [8]:
from flair.models import SequenceTagger

# Load the trained model
model = SequenceTagger.load('output/best-model.pt')

# Assuming you've already created a corpus with test data
# test_data = corpus.test

2023-12-16 19:15:21,760 SequenceTagger predicts: Dictionary with 11 tags: O, B-Requirement, I-Requirement, B-Application, I-Application, B-Exception, I-Exception, B-Selection, I-Selection, <START>, <STOP>


In [11]:
from flair.data import Sentence

# Example: Predicting tags for a single sentence
sentence = Sentence('''Maximum travel distances Location Where travel is possible in one direction only (m) Where travel is possible in more than one direction (m) Places of special fire hazard 1 9 1 18 Areas with seating in rows 1 15 1 32 Areas not listed above 1 18 1 45 Ground storey of small premises with a single exit 1 27 N/A Notes: 1''')
model.predict(sentence)
print(sentence.to_tagged_string())


Sentence[70]: "Maximum travel distances Location Where travel is possible in one direction only (m) Where travel is possible in more than one direction (m) Places of special fire hazard 1 9 1 18 Areas with seating in rows 1 15 1 32 Areas not listed above 1 18 1 45 Ground storey of small premises with a single exit 1 27 N/A Notes: 1" → ["Maximum travel distances Location Where travel is possible in one direction only (m) Where travel is possible in more than one direction (m) Places of special fire hazard 1 9"/Requirement, "1"/Exception, "18"/Exception, "Areas with seating in rows 1"/Requirement, "15"/Exception, "1"/Exception, "32"/Exception, "Areas not listed above 1 18 1 45 Ground storey of small premises with a single exit 1 27 N/A Notes: 1"/Requirement]


In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence
import os
import pathlib
import sys
import re

temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

sys.stdin.reconfigure(encoding='utf-8')
sys.stdout.reconfigure(encoding='utf-8')

def infer(text):
    # Load the trained model
    path = f'best-model.pt'
    model = SequenceTagger.load(path)

    text_split = text.split(".")

    final_string = ""
    a = []
    for x in text_split:
        sentence = Sentence(x)
        model.predict(sentence)
        value = sentence.to_tagged_string()
        a.append(value)
        value_split = value.split('\u2192')[-1]



        pattern = r'"([^"]*)"/\w+'
        extracted_items = re.findall(pattern, value_split)

        pattern_2 = r'"/([A-Z][a-z]*)'
        extracted_items_tags =  re.findall(pattern_2, value_split)


        for i, item in enumerate(extracted_items):
            final_string += f"<strong>{extracted_items_tags[i]}</strong>: {item}<br>"


    return final_string



In [ ]:
print(infer(""))